In [1]:
%cd /content/drive/MyDrive/데청캠 5조/Test

/content/drive/MyDrive/데청캠 5조/Test


In [2]:
!pwd

/content/drive/MyDrive/데청캠 5조/Test


## 전처리 1

In [3]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import time
from tqdm import notebook
import re

In [4]:
# Whole_MOV_List 함수 : 전체 동영상 리스트 받아오기
# mp4 파일이 들어있는 폴더명 입력
def Whole_File_List(Directory):
    File_List = os.listdir(Directory)

    # ./MOV/파일이름.mp4 형태로 파일명 변형
    Using_File = np.array([Directory+'/'+i for i in File_List])
    
    return Using_File

In [5]:
# Two_min_under_MOV 함수 : 2분 이하 동영상 선별 

def Two_min_under_MOV(MOV_List):
    
    # Using_List : 사용할 영상들 file_name 모음 
    Using_List = MOV_List[:]
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        clip = VideoFileClip(MOV_List[i])
        play_time = clip.duration
        
        # 재생 시간이 2분을 넘는 경우 -> Using_list에서 제외
        if play_time > 60*2 :
            print('Except MOV_file name :', Using_List[i])
            del_index = np.append(del_index, [i])
            
    Using_List = np.delete(Using_List, del_index)
            
    # 2분 미만인 동영상 리스트
    return Using_List

In [6]:
# 전처리 1-1 : 앵커 1명의 얼굴로 시작되는 영상 리스트 찾기

def one_anchor_starts_MOV(MOV_List):
     
    Using_MOV = MOV_List[:]
    
    # 파일 확장자를 제외한 순수한 파일명
    Origin_file_name = np.array([])
    Origin_file_name = Origin_file_name.astype(str)
    
    # 가중치 파일 경로
    cascade_filename = 'haarcascade_frontalface_alt.xml'
    # 모델 불러오기
    cascade = cv2.CascadeClassifier(cascade_filename)
    
    # 첫 장면에서 인식한 얼굴의 수
    face_num_list = np.array([])
    face_num_list = face_num_list.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        video = cv2.VideoCapture(MOV_List[i]) #video name change
        ret, frame = video.read()
        
        # 첫 번째로 등장하는 프레임만 취급
        frame[0]

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        results = cascade.detectMultiScale(gray,
                                          scaleFactor=1.2,
                                          minNeighbors=5, minSize=(20,20))
        
        face_num_list = np.append(face_num_list, [len(results)])
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    # 앵커 1명으로 시작하는 영상 찾기
    for k in range(len(face_num_list)):
        if face_num_list[k]!=1:
            del_index = np.append(del_index, [k])
    
    Using_MOV = np.delete(Using_MOV, del_index)
    
    for MOV in Using_MOV:
        MOV_origin = re.sub(pattern='.\/MOV\/', repl='', string=MOV)
        MOV_origin = re.sub(pattern='.mp4', repl='', string=MOV_origin)
        
        Origin_file_name = np.append(Origin_file_name, MOV_origin)

    # 파일 처리 쉽게 할 수 있는 것을 목표 
    # -> MOV 원본 파일 주소, MOV 파일확장자 지운 버전 모두 제공 
    return Using_MOV, Origin_file_name

## 전처리 2

In [7]:
!pip3 install pydub

In [8]:
!pip install speechrecognition

In [9]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import speech_recognition as sr

In [10]:
#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

In [11]:
def video_segment(File_List):

    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300

    for File in notebook.tqdm(File_List):
        print(File)
        clip = mp.VideoFileClip(File)
        
        MOV = re.sub(pattern='./MOV/', repl='', string=File)
        MOV = re.sub(pattern='.mp4', repl='', string=MOV)

        sound_file_name = './CUT_VID/' + MOV + '.wav'
        clip.audio.write_audiofile(sound_file_name)

        audio_segment = AudioSegment.from_wav(sound_file_name)

#normalize audio_segment to -20dBFS 
        normalized_sound = match_target_amplitude(audio_segment, -20.0)
        print("length of audio_segment={} seconds".format(len(normalized_sound)/1000))

#Print detected non-silent chunks, which in our case would be spoken words.
        nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=300, silence_thresh=-20, seek_step=1)

        a = 1

#convert ms to seconds
        print('start,Stop')
        for chunks in nonsilent_data:
            time_line = [[chunk/1000 for chunk in chunks]]
            for i in time_line:
                print(i[0], i[1])
                
                # [ 이슈 발생 해결 ]
                # mp4 -> mp4 과정에서 화면 안 나오는 현상 해결
                # wav -> wav로 변환시키는 과정에서 소리 유실되는 현상 해결 
                b = str(a)
                # ffmpeg_extract_subclip(File, i[0], i[1], targetname='./SEG_VID/'+MOV+'_'+b+'.mp4')
                video_clip = clip.subclip(i[0], i[1])
                video_clip.write_videofile('./SEG_VID/'+MOV+'_'+b+'.mp4')
                ffmpeg_extract_subclip(sound_file_name, i[0], i[1], targetname='./SEG_VID/'+MOV+'_'+b+'.wav')

                audio_name='./SEG_VID/' + MOV + '_' + b + '.wav'

                audio = sr.AudioFile(audio_name)

                with audio as source:
                    audio_news = recognizer.record(source)

                text = recognizer.recognize_google(audio_data=audio_news, language="ko-KR", show_all=True)

# 문제 사항 : 대부분의 stt가 문맥에 맞춰 해석하는 경우가 많아서 제대로 된 txt를 뽑아내기 힘듦...
                f = open('./SEG_VID/' + MOV + '_' + b + '.txt', 'w')
                text = str(text)
                f.write(text)
                f.close()
                
                a += 1
    
    return './SEG_VID' #잘린 비디오 디렉토리

In [12]:
import cv2
import glob
import dlib
import numpy as np
import os
import imutils
from matplotlib import pyplot as plt
from imutils import face_utils

#전처리 1-2
#앵커 얼굴이 1명만 화면 상에 나타날때 까지의 영상을 자른다.

def one_anchor_only (Origin_File_List):
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    for Origin_File in Origin_File_List:
        MOV_name = './MOV/' + Origin_File + '.mp4'

        video =  cv2.VideoCapture(MOV_name)

        fps = video.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
        height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
        size = (round(width), round(height))
        out = cv2.VideoWriter('./One_Anch/%s.mp4' %Origin_File, fourcc, fps, size)

        while(video.isOpened()):
        
            ret, frame = video.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 
                                                  scaleFactor=1.2, 
                                                  minNeighbors=5, 
                                                  minSize=(20,20))

            if len(faces) == 1:
                out.write(frame)
            else:
                break

        out.release()
    
    return './One_Anch' #자른 영상들이 있는 디렉토리 반환

In [13]:
# [ 발견한 사항 ] 
# (인식된 얼굴 수) = (1, 1, 1, 1, 0, 1, 1, 1) 형태라고 가정하자.
# 저기서는 0이라고 인식되었지만, 실제로는 1명 만큼의 얼굴이 출력되는 상태임.
# 이런 상황에서는 데이터 손실이 발생함.

# [ 보완 아이디어 ]
# 0이 나온 자리를 기준으로 앞/뒤에서 연속으로 1이 각각 N번 (ex. 3번) 나올 경우
# -> 0을 1로 바꿔주는 작업을 해보는 것을 어떨까?
# -> 문제는 이럴 경우에 frame 좌표와 짝지어주는 작업을 추가적으로 해야할 것 같음...

# 일단 이 점은 자고 일어나서 시도해봐야겠음! 
# 꿈 속에서 알고리즘 뚝딱 생각나면 좋겠다!!

## 전처리 3

In [14]:
import cv2
import glob
import dlib
import numpy as np
import os
import imutils
from matplotlib import pyplot as plt
from imutils import face_utils

In [15]:
def face_crop_vid (File_List):
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    for File in notebook.tqdm(File_List):
        if File[-3:] == 'mp4':
          MOV = re.sub(pattern='.mp4', repl='', string=File)
          MOV = re.sub(pattern='./\w+/', repl='', string=MOV)
          print(MOV)

          video =  cv2.VideoCapture(File)

          fps = video.get(cv2.CAP_PROP_FPS)
          fourcc = cv2.VideoWriter_fourcc(*'DIVX')
          width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
          height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
          size = (round(width), round(height))
          out = cv2.VideoWriter('./FINAL_VID/%s.mp4' %MOV, fourcc, fps, size)

          while(video.isOpened()):
          
              ret, frame = video.read()
              
              if frame is None:
                  break

              gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              faces = face_cascade.detectMultiScale(gray, 
                                                    scaleFactor=1.2, 
                                                    minNeighbors=5, 
                                                    minSize=(20,20))


              for (x, y, w, h) in faces:
                  cropped = frame[y - int(h/4):y + h + int(h/4), x - int(w/4):x + w + int(w/4)]


              out.write(cropped)

          out.release()
    return './FINAL_VID'

In [16]:
from moviepy.editor import *
import re

In [17]:
def concat_vid (MOV_Directory, Audio_Directory):

    MOV_List = os.listdir(MOV_Directory)

    AUD_List = glob.glob(Audio_Directory + '*.wav')
    # AUD_List = os.listdir(AUD_List)

    for MOV in MOV_List:
        for AUD in AUD_List:
            mov = re.sub(pattern='.mp4', repl='', string=MOV)
            aud = re.sub(pattern='.wav', repl='', string=AUD)

            if mov == aud:
                MOV_name = MOV_Directory + '/' + MOV
                AUD_name = Audio_Directory + '/' + AUD
                videoclip = VideoFileClip(MOV_name)
                audioclip = AudioFileClip(AUD_name)
                new_audioclip = CompositeAudioClip([audioclip])
                videoclip.audio = new_audioclip
                videoclip.write_videofile(MOV_name)

In [18]:
from moviepy.editor import VideoFileClip

def get_duration(filename):
    clip = VideoFileClip(filename)
    return clip.duration

In [19]:
mov = Whole_File_List('./MOV') #전체 영상 리스트

mov_list = Two_min_under_MOV(mov) #2분 이하 영상들

Except MOV_file name : ./MOV/readnaversid1750oid448aid0000195268.mp4



In [20]:
mov_list

array(['./MOV/video.mp4', './MOV/readnaversid1750oid448aid0000195576.mp4',
       './MOV/readnaversid1750oid448aid0000196603.mp4',
       './MOV/readnaversid1750oid448aid0000310334.mp4'], dtype='<U45')

In [21]:
Using_MOV, one_anchor_s = one_anchor_starts_MOV(mov_list) #앵커 1명으로 시작하는 영상 리스트b

In [22]:
Using_MOV

array(['./MOV/readnaversid1750oid448aid0000195576.mp4',
       './MOV/readnaversid1750oid448aid0000310334.mp4'], dtype='<U45')

In [23]:
one_anchor_s

array(['readnaversid1750oid448aid0000195576',
       'readnaversid1750oid448aid0000310334'], dtype='<U35')

In [24]:
one_anchor_cut = one_anchor_only(one_anchor_s) #앵커 한 명만 말하는 부분 자른 영상 디렉토리 반환

In [25]:
b = Whole_File_List('./One_Anch')
b

array(['./One_Anch/readnaversid1750oid448aid0000195576.mp4',
       './One_Anch/readnaversid1750oid448aid0000310334.mp4'], dtype='<U50')

In [26]:
# i는 원본 영상
for i in Using_MOV:
  # j는 앵커 1명 시점까지만 자른 영상
    for j in b:
      origin_i = re.sub(pattern='./MOV/', repl='', string=i)
      origin_j = re.sub(pattern='./One_Anch/', repl='', string=j)

      if origin_i == origin_j :
        clip = VideoFileClip(i).subclip(0, get_duration(j))
        clip.write_videofile("./CUT_VID/"+origin_i)
        # ffmpeg_extract_subclip(i, 0, get_duration(j), targetname="./CUT_VID/"+origin_i)

#CUT_VID라는 폴더에 사람이 한명만 있는 부분만 잘린 영상들이 저장되어 있음

[MoviePy] >>>> Building video ./CUT_VID/readnaversid1750oid448aid0000195576.mp4
[MoviePy] Writing audio in readnaversid1750oid448aid0000195576TEMP_MPY_wvf_snd.mp3


100%|██████████| 79/79 [00:00<00:00, 758.67it/s]

[MoviePy] Done.
[MoviePy] Writing video ./CUT_VID/readnaversid1750oid448aid0000195576.mp4



100%|██████████| 107/107 [00:01<00:00, 71.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./CUT_VID/readnaversid1750oid448aid0000195576.mp4 

[MoviePy] >>>> Building video ./CUT_VID/readnaversid1750oid448aid0000310334.mp4
[MoviePy] Writing audio in readnaversid1750oid448aid0000310334TEMP_MPY_wvf_snd.mp3


100%|██████████| 11/11 [00:00<00:00, 1201.15it/s]

[MoviePy] Done.
[MoviePy] Writing video ./CUT_VID/readnaversid1750oid448aid0000310334.mp4



100%|██████████| 15/15 [00:00<00:00, 80.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./CUT_VID/readnaversid1750oid448aid0000310334.mp4 



In [27]:
seg_vid = video_segment(Using_MOV)
#seg_vid에 문장 단위 영상이 존재

./MOV/readnaversid1750oid448aid0000195576.mp4
[MoviePy] Writing audio in ./CUT_VID/readnaversid1750oid448aid0000195576.wav


100%|██████████| 1677/1677 [00:00<00:00, 1806.74it/s]

[MoviePy] Done.


length of audio_segment=76.01 seconds
start,Stop
0.426 0.93
[MoviePy] >>>> Building video ./SEG_VID/readnaversid1750oid448aid0000195576_1.mp4
[MoviePy] Writing audio in readnaversid1750oid448aid0000195576_1TEMP_MPY_wvf_snd.mp3


100%|██████████| 12/12 [00:00<00:00, 274.29it/s]

...

100%|██████████| 1/1 [00:00<00:00, 160.11it/s]

[MoviePy] Done.
[MoviePy] Writing video ./SEG_VID/readnaversid1750oid448aid0000310334_70.mp4



100%|██████████| 1/1 [00:00<00:00, 80.38it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: ./SEG_VID/readnaversid1750oid448aid0000310334_70.mp4 


[MoviePy] Running:
>>> /root/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1 -y -i ./CUT_VID/readnaversid1750oid448aid0000310334.wav -ss 91.44 -t 0.01 -vcodec copy -acodec copy ./SEG_VID/readnaversid1750oid448aid0000310334_70.wav


... command successful.



In [28]:
seg_vid = Whole_File_List('./SEG_VID')

In [29]:
seg_vid[0:5]

array(['./SEG_VID/readnaversid1750oid448aid0000195576_1.mp4',
       './SEG_VID/readnaversid1750oid448aid0000195576_1.wav',
       './SEG_VID/readnaversid1750oid448aid0000195576_1.txt',
       './SEG_VID/readnaversid1750oid448aid0000195576_2.mp4',
       './SEG_VID/readnaversid1750oid448aid0000195576_2.wav'],
      dtype='<U52')

In [30]:
final_vid = face_crop_vid (seg_vid)
#final_vid에 얼굴이 crop된 영상 존재
# !!! 파일이 손상되었다고 나옴 !!! -> 코드는 돌아가지만 파일이 이상한 상태

readnaversid1750oid448aid0000195576_1
readnaversid1750oid448aid0000195576_2
readnaversid1750oid448aid0000195576_3
readnaversid1750oid448aid0000195576_4
readnaversid1750oid448aid0000195576_5
readnaversid1750oid448aid0000195576_6
readnaversid1750oid448aid0000195576_7
readnaversid1750oid448aid0000195576_8
readnaversid1750oid448aid0000195576_9
readnaversid1750oid448aid0000195576_10
readnaversid1750oid448aid0000195576_11
readnaversid1750oid448aid0000195576_12
readnaversid1750oid448aid0000195576_13
readnaversid1750oid448aid0000195576_14
readnaversid1750oid448aid0000195576_15
readnaversid1750oid448aid0000195576_16
readnaversid1750oid448aid0000195576_17
readnaversid1750oid448aid0000195576_18
readnaversid1750oid448aid0000195576_19
readnaversid1750oid448aid0000195576_20
readnaversid1750oid448aid0000195576_21
readnaversid1750oid448aid0000195576_22
readnaversid1750oid448aid0000195576_23
readnaversid1750oid448aid0000195576_24
readnaversid1750oid448aid0000195576_25
readnaversid1750oid448aid000019557

In [31]:
concat_vid ('./FINAL_VID', './SEG_VID')
#seg_vid에 존재하는 audio 파일과 final_vid에 존재하는 얼굴 crop video 합치기
#합친 파일은 final_vid에 존재